In [1]:
from IPython.display import display, HTML
import json
from pprint import pprint
import sys
import time
import requests
from datetime import datetime, timedelta
import pandas as pd
import csv
import ast
import os
import ssl
from pathlib import Path
from statistics import mean
import os.path
sys.path.append('utils/')
import general_splits
import splits_espn
import defense_stats_vs_position
sys.path.append('../')
sys.path.append('../utils/')
sys.path.append('../predictor/utils/')
sys.path.append('../predictor/utils/calls_stats_nba/')
import Dunkest_BOT

In [2]:
class Archiver:
    def __init__(self):
        self.PATH_DATA = 'data/'
        self.PATH_DATA_FIXED = 'data/fixed/'
        self.PATH_FOLDER_DATA = None
        self.prefix_files = None
        with open(self.PATH_DATA_FIXED + 'config.json') as config:
            self.config = json.load(config)
        self.next_game_dict = {}
        self.players_filtered = {}
        with open(self.PATH_DATA_FIXED + 'players_nba.json') as players_nba:
            self.players_name_id = json.load(players_nba)
        with open(self.PATH_DATA_FIXED
                  + 'players_nba_espn.json') as players_nba_espn:
            self.players_name_id_espn = json.load(players_nba_espn)
        with open(self.PATH_DATA_FIXED + 'full_schedule.json') as schedule:
            self.full_schedule = json.load(schedule)
        with open(self.PATH_DATA_FIXED + 'teams_nba.json') as teams:
            self.teams_name_id = json.load(teams)
        self.opponent_stats_G = {}
        self.opponent_stats_F = {}
        self.opponent_stats_C = {}
        self.predictions_rows_dict = {}
        self.teams_play_next_day = {}
        self.players_filtered = {}
        self.splits_players = {}
        
    
    def make_data_folder(self):
        now = datetime.today()
        name_folder = '{}_{}_{}/'.format(now.day, now.month, now.year)
        self.prefix_files = '{}_{}_{}-'.format(now.day, now.month, now.year)
        if not os.path.isdir(self.PATH_DATA + name_folder):
            os.mkdir(self.PATH_DATA + name_folder)
        self.PATH_FOLDER_DATA = self.PATH_DATA + name_folder
        
    def compute_next_game_all_teams(self):
        self.make_data_folder()
        for team, id_team in self.teams_name_id.items():
            games_team = self.full_schedule[team]
            now = datetime.now()
            minimium_difference = now - datetime(2000, 1, 1, 1, 00)
            next_game = None
            for game in games_team:
                # print(game)
                game_time = datetime.strptime(game[2], '%Y-%m-%d %H:%M:%S')
                if game_time - now > pd.Timedelta(0):
                    if game_time - now < minimium_difference:
                        minimium_difference = game_time - now
                        next_game = game
            self.next_game_dict[team] = next_game
        with open(self.PATH_FOLDER_DATA + self.prefix_files + 'next_game_teams.json',
                  'w') as outfile:
            json.dump(self.next_game_dict, outfile)
            
    def get_list_teams_next_day(self):
        now = datetime.today()
        for team, game in self.next_game_dict.items():
            diff = datetime.strptime(game[2], '%Y-%m-%d %H:%M:%S') - now
            if diff.days == 0:
                if diff.seconds / 3600 < 22:
                    self.teams_play_next_day[team] = self.next_game_dict[team]
        with open(self.PATH_FOLDER_DATA + self.prefix_files + 'next_day_games_teams.json',
                  'w') as outfile:
            json.dump(self.teams_play_next_day, outfile)
            
    def filter_players(self, players):
        for p,p_data in players.items():
            if p_data[0] in self.teams_play_next_day:
                self.players_filtered[p] = p_data
            
    def get_players_splits(self):
        season = self.config['SEASON']
        if os.path.isfile(self.PATH_FOLDER_DATA + self.prefix_files + 'full_players_splits.json'):
            os.remove(self.PATH_FOLDER_DATA + self.prefix_files + 'full_players_splits.json')
        total_players = len(self.players_filtered)
        index = 1
        for player, data_player in self.players_filtered.items():
            print(player, str(index) + ' of ' + str(total_players))
            next_game_player = self.teams_play_next_day[data_player[0]]
            print("next game player: {}".format(next_game_player))
            days_rest_player = next_game_player[3]
            id_player = self.players_name_id_espn.get(player, None)
            print("id player: {}".format(id_player))
            if id_player is None:
                index += 1
                continue
            splits_player = splits_espn.get_splits_player(id_player)
            if splits_player is None or len(splits_player) == 0:
                index += 1
                continue
            vs_team = next_game_player[1]
            print("vs: {}".format(vs_team))
            position_player = data_player[1]
            where_to_play = next_game_player[0]
            self.splits_players[player] = splits_player
            splits_to_write = {player: splits_player}
            if os.path.isfile(self.PATH_FOLDER_DATA + self.prefix_files + 'full_players_splits.json'):
                with open(self.PATH_FOLDER_DATA + self.prefix_files + 'full_players_splits.json',
                          'a') as outfile:
                    json.dump(splits_to_write, outfile)
            else:
                with open(self.PATH_FOLDER_DATA + self.prefix_files + 'full_players_splits.json',
                          'w') as outfile:
                    json.dump(splits_to_write, outfile)
            index += 1
            print("-" * 75)
        data_ = None
        #adjustment of json file 
        with open(archiver.PATH_FOLDER_DATA + archiver.prefix_files + 'full_players_splits.json', 'r') as file:
            data_ = file.read().replace('}}{', '},')
        with open(archiver.PATH_FOLDER_DATA + archiver.prefix_files + 'full_players_splits.json', 'w') as file:
            file.write(data_)

In [3]:
"""
- Calcolo le partite della prossima notte -> lista di teams che giocano
- Calcolo lista di giocatori che giocheranno la prossima notte, guardando la lista di partite
  appena trovata e filtrando per i giocatori non infortunati
- Per ognuno di questi giocatori prendo gli splits
- Scarico le stats opponent per ruolo
- Salvo tutto, aggiungendo giorno mese e anno al nome file
"""
print("Instancing archiver")
archiver = Archiver()
print("Computing the next game of every team")
archiver.compute_next_game_all_teams()
print("Keeping only the games of next night")
archiver.get_list_teams_next_day()

Instancing archiver
Computing the next game of every team
Keeping only the games of next night


In [4]:
print("Instancing Dunkest BOT")
bot = Dunkest_BOT.dunkest_bot()
print("Downloading players data from dunkest")
bot.download_dunkest_players()

Instancing Dunkest BOT
Bot started
--------------------------------------


In [5]:
print("Filtering players who will play the next night")
archiver.filter_players(bot.players)

Filtering players who will play the next night


In [6]:
pprint(archiver.players_filtered)

{'abdel nader': ['oklahoma city thunder', 'A', 4.3, 0.0, 0.0],
 'alex caruso': ['los angeles lakers', 'G', 5.4, 0.0, 0.0],
 'alfonzo mckinnie': ['cleveland cavaliers', 'A', 4, 0.0, 0.0],
 'allonzo trier': ['new york knicks', 'G', 4.2, 0.0, 0.0],
 'andre drummond': ['detroit pistons', 'C', 17, 0.0, 0.0],
 'andrew wiggins': ['minnesota timberwolves', 'A', 9, 0.0, 0.0],
 'anfernee simons': ['portland trail blazers', 'G', 4.4, 0.0, 0.0],
 'ante zizic': ['cleveland cavaliers', 'C', 4, 0.0, 0.0],
 'anthony davis': ['los angeles lakers', 'A', 22.8, 0.0, 0.0],
 'anthony tolliver': ['portland trail blazers', 'A', 4, 0.0, 0.0],
 'aron baynes': ['phoenix suns', 'C', 8, 0.0, 0.0],
 'bobby portis': ['new york knicks', 'A', 7.7, 0.0, 0.0],
 'brandon clarke': ['memphis grizzlies', 'A', 8, 0.0, 0.0],
 'brandon knight': ['cleveland cavaliers', 'G', 4, 0.0, 0.0],
 'bruce brown': ['detroit pistons', 'G', 6.8, 0.0, 0.0],
 'bruno caboclo': ['memphis grizzlies', 'A', 4, 0.0, 0.0],
 'buddy hield': ['sacramen

In [7]:
print("Downloading the splits of every filtered payer")
archiver.get_players_splits()

abdel nader 1 of 162
next game player: ['away', 'brooklyn nets', '2020-01-08 01:30:00', 0]
id player: 2595435
vs: brooklyn nets
---------------------------------------------------------------------------
alex caruso 2 of 162
next game player: ['home', 'new york knicks', '2020-01-08 04:30:00', 1]
id player: 2991350
vs: new york knicks
---------------------------------------------------------------------------
alfonzo mckinnie 3 of 162
next game player: ['home', 'detroit pistons', '2020-01-08 01:00:00', 1]
id player: 2530923
vs: detroit pistons
---------------------------------------------------------------------------
allonzo trier 4 of 162
next game player: ['away', 'los angeles lakers', '2020-01-08 04:30:00', 2]
id player: 3907525
vs: los angeles lakers
---------------------------------------------------------------------------
andre drummond 5 of 162
next game player: ['away', 'cleveland cavaliers', '2020-01-08 01:00:00', 1]
id player: 6585
vs: cleveland cavaliers
-------------------

vs: new york knicks
---------------------------------------------------------------------------
dennis schroder 44 of 162
next game player: ['away', 'brooklyn nets', '2020-01-08 01:30:00', 0]
id player: 3032979
vs: brooklyn nets
---------------------------------------------------------------------------
deonte burton 45 of 162
next game player: ['away', 'brooklyn nets', '2020-01-08 01:30:00', 0]
id player: 3059358
vs: brooklyn nets
---------------------------------------------------------------------------
derrick rose 46 of 162
next game player: ['away', 'cleveland cavaliers', '2020-01-08 01:00:00', 1]
id player: 3456
vs: cleveland cavaliers
---------------------------------------------------------------------------
devin booker 47 of 162
next game player: ['home', 'sacramento kings', '2020-01-08 03:00:00', 1]
id player: 3136193
vs: sacramento kings
---------------------------------------------------------------------------
dewayne dedmon 48 of 162
next game player: ['away', 'phoenix 

vs: los angeles lakers
---------------------------------------------------------------------------
justin patton 88 of 162
next game player: ['away', 'brooklyn nets', '2020-01-08 01:30:00', 0]
id player: 3912332
vs: brooklyn nets
---------------------------------------------------------------------------
justin james 89 of 162
next game player: ['away', 'phoenix suns', '2020-01-08 03:00:00', 0]
id player: None
kadeem allen 90 of 162
next game player: ['away', 'los angeles lakers', '2020-01-08 04:30:00', 2]
id player: None
keita bates-diop 91 of 162
next game player: ['away', 'memphis grizzlies', '2020-01-08 02:00:00', 1]
id player: 3136779
vs: memphis grizzlies
---------------------------------------------------------------------------
kelan martin 92 of 162
next game player: ['away', 'memphis grizzlies', '2020-01-08 02:00:00', 1]
id player: None
kelly oubre jr jr 93 of 162
next game player: ['home', 'sacramento kings', '2020-01-08 03:00:00', 1]
id player: None
kent bazemore 94 of 162


vs: brooklyn nets
---------------------------------------------------------------------------
solomon hill 135 of 162
next game player: ['home', 'minnesota timberwolves', '2020-01-08 02:00:00', 1]
id player: 2488958
vs: minnesota timberwolves
---------------------------------------------------------------------------
spencer dinwiddie 136 of 162
next game player: ['home', 'oklahoma city thunder', '2020-01-08 01:30:00', 0]
id player: 2580782
vs: oklahoma city thunder
---------------------------------------------------------------------------
stanley johnson 137 of 162
next game player: ['home', 'portland trail blazers', '2020-01-08 01:00:00', 2]
id player: 3134881
vs: portland trail blazers
---------------------------------------------------------------------------
steven adams 138 of 162
next game player: ['away', 'brooklyn nets', '2020-01-08 01:30:00', 0]
id player: 2991235
vs: brooklyn nets
---------------------------------------------------------------------------
svi mykhailiuk 139

In [8]:
defense_stats_vs_position.main()

In [9]:
#just to check
a = 0
for p,s in archiver.splits_players.items():
    if s.get('away',None) is None:
        print(p)
        a += 1
print(a)

kostas antetokounmpo
1


In [10]:
list_players_dunkest = requests.get(
        'https://api.dunkest.com/api/players?league_id=1',
        verify=False)
list_players_dunkest = json.loads(list_players_dunkest.text)

In [11]:
popularity_dict = {}
players_data_ = {}
for elem in list_players_dunkest:
    players_data_[elem['name'].lower() + ' ' + elem['surname'].lower()] = elem
    popularity_dict[elem['name'].lower() + ' ' + elem['surname'].lower()] = \
    [elem['quotation'],elem['avg_score'],elem['popularity']]

In [12]:
df = pd.DataFrame.from_dict(popularity_dict, orient='index', columns=['quotation','avg score','popularity'])

In [13]:
df["hot index"] = df.apply(
    lambda row: row['avg score'] * row['popularity'] / row['quotation'], axis=1
)

In [14]:
df.sort_values(by='hot index', ascending=False).head(50).sort_values(by='quotation', ascending=True)

,quotation,avg score,popularity,hot index
kostas antetokounmpo,4.0,3.00,10.1,7.575000
antetokounmpo thanasis,4.0,5.14,9.9,12.721500
tacko fall,4.0,6.00,13.1,19.650000
kyle o'quinn,4.2,8.60,5.2,10.647619
isaac bonga,4.2,7.13,7.6,12.901905
bryn forbes,5.0,9.62,4.4,8.465600
kyle korver,5.0,8.50,4.6,7.820000
shabazz napier,5.1,11.64,3.1,7.075294
ian mahinmi,5.4,14.64,2.8,7.591111
jordan mcrae,5.5,15.13,2.9,7.977636


In [15]:
#attach dunkest id to players data
players_with_id = {}
response = requests.get(
            'https://api.dunkest.com/api/players?league_id=1', verify=False)
json_data = json.loads(response.text)
for player in json_data:
    first_name_player = player["name"]
    surname_player = player["surname"]
    name_player = (first_name_player +
                   " " + surname_player).strip().lower().replace(
                       ".", "").replace('byombo', 'biyombo').replace(
                           'malcom', 'malcolm').replace(
                               'mohamed bamba', 'mo bamba').replace(
                                   'hilario nene',
                                   'nene hilario').replace(
                                       'kelly oubre',
                                       'kelly oubre jr').replace(
                                           'edrice adebayo',
                                           'bam adebayo')
    if player['role_tag'] != "COACH":
        if player['lineup_tag'] != 'QUESTIONABLE' and player['lineup_tag'] != 'INJURED' and player[
                'lineup_tag'] != 'DISQUALIFIED' and 'IN_DOUBT' not in player['lineup_tag']:
            players_with_id[name_player] = bot.players[name_player] + [player['id']] \
            + [player['avg_score'],player['popularity']]

In [16]:
#getting players score of all rounds from dunkest
players_with_id_ = {}
for p,p_data in players_with_id.items():
    print("="*60)
    print(p, p_data)
    url = 'https://api.dunkest.com/api/league/1/fanta_player_stats_detail?league_id=1&player_id='\
            + str(p_data[5])
    response = requests.get(url, verify=False)
    json_score_p = json.loads(response.text) #list of scores for every round
    #pprint(json_score_p)
    round_scores = []
    for round_data in json_score_p:
        #print(round_data)
        #print('='*60)
        #print(round_data)
        if len(round_data) == 0:
            round_scores.append(float(0))
        else:
            round_scores.append(float(round_data['total_score']))
    players_with_id_[p] = players_with_id[p]
    temp = players_with_id[p]
    temp.append(round_scores)
    players_with_id_[p] = temp
    print(players_with_id_[p])

aaron gordon ['orlando magic', 'A', 11.7, 0.0, 0.0, 1049, 19.12, 0.9]
['orlando magic', 'A', 11.7, 0.0, 0.0, 1049, 19.12, 0.9, [14.5, 14.5, 5.8, 7.2, 15.0, 22.2, 12.2, 16.8, 18.2, 0.0, 0.0, 0.0, 11.8, 10.8, 47.0, 18.5, 29.2, 23.8, 4.8, 18.0, 34.5, 30.5, 0.0, 23.0, 23.2]]
aaron holiday ['indiana pacers', 'G', 7.3, 0.0, 0.0, 1470, 14.8, 1.5]
['indiana pacers', 'G', 7.3, 0.0, 0.0, 1470, 14.8, 1.5, [-5.5, 0.0, 0.0, 3.0, 13.0, 17.0, 15.8, 11.8, 47.0, 15.8, 22.2, -0.5, 4.5, 11.0, 2.0, 15.5, 26.5, 14.0, 27.2, 20.0, 11.2, 23.5, 20.2, 12.0, 13.2]]
abdel nader ['oklahoma city thunder', 'A', 4.3, 0.0, 0.0, 1533, 7.25, 0.8]
['oklahoma city thunder', 'A', 4.3, 0.0, 0.0, 1533, 7.25, 0.8, [0.0, -4.5, 0.2, 5.0, 0.0, 0.0, 0.0, 5.5, 6.2, 0.0, 5.5, 21.2, 23.0, 2.0, 5.0, 8.2, 13.2, 12.5, 6.0, 5.0, 13.5, 2.5, 5.5, 2.0, 0.0]]
admiral schofield ['washington wizards', 'A', 4, 0.0, 0.0, 2987, 3.95, 0.9]
['washington wizards', 'A', 4, 0.0, 0.0, 2987, 3.95, 0.9, [-4.5, 15.5, 0.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0

['houston rockets', 'G', 6, 0.0, 0.0, 1148, 8.84, 0.4, [-2.0, 9.2, 4.0, 4.0, 3.8, 12.2, 8.8, 16.8, 7.0, 3.5, -0.8, 7.0, 22.8, 19.5, 19.0, 12.0, 0.0, 3.0, 7.0, 12.0, 8.0, 12.5, 6.0, 8.0, 0.0]]
bam adebayo ['miami heat', 'C', 16.2, 0.0, 0.0, 2934, 34.28, 12.8]
['miami heat', 'C', 16.2, 0.0, 0.0, 2934, 34.28, 12.8, [27.0, 42.5, 41.0, 32.5, 19.2, 43.5, 32.5, 39.0, 27.5, 45.0, 41.8, 33.2, 16.0, 31.0, 7.5, 44.5, 63.5, 22.5, 29.5, 29.2, 45.2, 49.0, 23.8, 35.0, 35.5]]
ben simmons ['philadelphia 76ers', 'G', 18, 0.0, 0.0, 1033, 33.36, 2.3]
['philadelphia 76ers', 'G', 18, 0.0, 0.0, 1033, 33.36, 2.3, [35.5, 31.5, 23.2, 39.0, 16.5, 0.0, 0.0, 28.8, 36.0, 39.2, 32.2, 30.8, 21.8, 33.8, 29.0, 51.0, 20.0, 48.8, 25.8, 16.5, 58.0, 22.2, 20.2, 62.0, 45.5]]
ben mclemore ['houston rockets', 'G', 6.4, 0.0, 0.0, 1294, 11.24, 0.6]
['houston rockets', 'G', 6.4, 0.0, 0.0, 1294, 11.24, 0.6, [1.5, -0.5, 1.0, 6.5, 9.5, 5.2, 8.5, 18.8, 6.2, 13.2, -0.5, 10.5, 42.5, 6.5, 28.2, 22.0, 10.0, 22.0, 18.0, 12.5, 7.5, 4.0, 1

['boston celtics', 'G', 4, 0.0, 0.0, 2951, 3.4, 1, [-2.8, 0.0, 3.0, 5.5, 19.0, 4.0, 5.5, 4.0, -1.0, -2.0, 7.0, 1.0, 4.5, 2.0, 7.5, 4.5, 0.0, 3.9, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, -1.5]]
cedi osman ['cleveland cavaliers', 'A', 7, 0.0, 0.0, 1284, 10.6, 1]
['cleveland cavaliers', 'A', 7, 0.0, 0.0, 1284, 10.6, 1, [12.5, 0.0, 8.0, 11.5, 7.8, 20.2, 12.5, 8.2, 2.2, 20.2, 17.5, 6.2, 20.5, 3.5, 18.5, 0.5, 8.0, 0.0, 6.8, 16.0, 17.2, 14.2, 16.2, 16.5, 0.0]]
chandler hutchison ['chicago bulls', 'A', 4, 0.0, 0.0, 1500, 5.29, 0.8]
['chicago bulls', 'A', 4, 0.0, 0.0, 1500, 5.29, 0.8, [0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.5, 13.0, 4.0, 0.0, 2.5, -2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.0]]
chandler parsons ['atlanta hawks', 'A', 4, 0.0, 0.0, 1273, -0.08, 1.2]
['atlanta hawks', 'A', 4, 0.0, 0.0, 1273, -0.08, 1.2, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.0, 7.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.5, 0.0, 0.0, 0.0]]
charlie brown jr ['atlanta hawks', 'A', 4,

['new york knicks', 'G', 4.4, 0.0, 0.0, 1447, 6.42, 0.2, [0.0, 0.0, 0.0, 0.0, -5.5, 4.5, 5.5, 16.0, 10.0, 0.0, 3.0, 5.0, 4.0, 15.0, 2.5, 0.8, 4.2, 21.0, -4.0, 8.0, 14.5, 10.5, 7.0, 0.0, 0.0]]
daniel theis ['boston celtics', 'A', 8.2, 0.0, 0.0, 1374, 16.06, 0.8]
['boston celtics', 'A', 8.2, 0.0, 0.0, 1374, 16.06, 0.8, [0.8, 7.2, 24.2, 20.8, 18.5, 14.2, 26.5, 16.0, 25.8, 6.5, 0.0, 25.0, 0.8, 11.0, 24.8, 19.5, 6.0, 15.5, 14.5, 22.5, 11.2, 23.8, 12.0, 24.5, 14.0]]
danilo gallinari ['oklahoma city thunder', 'A', 10.4, 0.0, 0.0, 1078, 18.6, 3.4]
['oklahoma city thunder', 'A', 10.4, 0.0, 0.0, 1078, 18.6, 3.4, [23.0, 19.5, 11.8, 21.5, 12.0, 29.0, 11.2, 33.5, 4.8, 13.0, 26.5, 8.8, 29.8, 39.5, 20.5, 9.2, 12.5, -3.5, 31.0, 25.0, 0.0, 0.0, 0.0, 13.0, 17.8]]
danny green ['los angeles lakers', 'G', 8.3, 0.0, 0.0, 1270, 14.78, 2.6]
['los angeles lakers', 'G', 8.3, 0.0, 0.0, 1270, 14.78, 2.6, [33.8, 17.5, 10.5, 15.5, 3.5, 8.2, 0.8, 11.5, 27.2, 20.8, 18.5, 5.0, 16.5, 2.5, 13.2, 26.5, 18.0, 10.0, 17.2, 

['los angeles clippers', 'G', 4.1, 0.0, 0.0, 3120, 5.04, 0.8, [0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.0, 6.5, 2.2, 11.8, 3.0, 15.0, 5.5, 0.0, 3.0, -3.0, 15.0, 2.5, 0.0]]
derrick favors ['new orleans pelicans', 'A', 10.9, 0.0, 0.0, 1097, 23.25, 1.3]
['new orleans pelicans', 'A', 10.9, 0.0, 0.0, 1097, 23.25, 1.3, [11.2, 19.2, 0.0, 0.0, -3.2, 10.5, 35.8, 47.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.2, 7.5, 24.0, 34.2, 32.0, 38.5, 38.2, 22.5]]
derrick jones jr ['miami heat', 'A', 5.8, 0.0, 0.0, 1442, 14.07, 2.9]
['miami heat', 'A', 5.8, 0.0, 0.0, 1442, 14.07, 2.9, [5.8, 19.5, 0.0, 0.0, 0.0, 8.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.8, 7.8, 17.8, 24.0, 16.5, 22.5, 13.0, 12.5, 13.0, 10.8, 27.0]]
derrick white ['san antonio spurs', 'G', 8.2, 0.0, 0.0, 1493, 13.95, 0.5]
['san antonio spurs', 'G', 8.2, 0.0, 0.0, 1493, 13.95, 0.5, [10.8, 13.0, 28.8, 22.5, 13.0, 12.5, 14.5, 4.0, 1.5, 0.0, 13.0, 11.5, 21.5, 8.5, -0.8, 12.4, 21.0, 7.5, 12.8, 26.0, 24.0, 9.2, 16

...

['new orleans pelicans', 'G', 4, 0.0, 0.0, 2942, 4.09, 0.4, [-0.8, 4.0, 23.5, 7.8, -5.0, 11.0, -6.0, 5.5, 22.0, 3.0, -2.5, 0.0, 0.0, 0.0, 2.5, 1.2, 2.0, 2.0, 0.0, -5.0, 1.0, 9.0, 0.0, 0.0, 2.5]]
nicolas batum ['charlotte hornets', 'A', 6.9, 0.0, 0.0, 1085, 10.43, 0.3]
['charlotte hornets', 'A', 6.9, 0.0, 0.0, 1085, 10.43, 0.3, [7.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.8, 13.8, 5.5, 23.2, 10.0, 1.8, 19.8, 0.0, 4.8, 0.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0]]
nicolas claxton ['brooklyn nets', 'A', 4, 0.0, 0.0, 2980, 2.39, 0.6]
['brooklyn nets', 'A', 4, 0.0, 0.0, 2980, 2.39, 0.6, [0.0, 0.0, 0.0, 0.0, 0.0, 14.8, 6.2, 0.0, 0.0, 2.5, 0.0, -2.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.0]]
nicolo' melli ['new orleans pelicans', 'A', 4, 0.0, 0.0, 831, 4.41, 2.7]
['new orleans pelicans', 'A', 4, 0.0, 0.0, 831, 4.41, 2.7, [14.5, 2.2, -1.2, 16.5, 0.0, -2.0, 0.0, 6.0, 24.0, 21.2, 1.0, 0.2, -5.5, -4.5, -0.5, -2.5, -5.0, 0.0, 0.0, 0.0, 5.5, 5.0, 0.0, 0.0, 0.0]]
nigel william-gos

['phoenix suns', 'G', 12.7, 0.0, 0.0, 1074, 25.41, 2.8, [33.0, 12.0, 24.8, 21.5, 42.5, 23.5, 50.0, 14.8, 0.0, 0.0, 0.0, 13.8, 18.0, 33.0, 12.5, 21.5, 19.0, 38.5, 30.2, 35.8, 29.0, 2.0, 44.0, 24.8, 15.0]]
rj barrett ['new york knicks', 'G', 6.5, 0.0, 0.0, 2933, 10.38, 1.9]
['new york knicks', 'G', 6.5, 0.0, 0.0, 2933, 10.38, 1.9, [19.5, 11.0, 36.0, 13.5, 11.8, 6.0, -3.0, 10.2, 17.8, 15.8, 0.0, 9.0, 13.5, 9.5, 12.2, 13.2, -3.0, 9.2, -2.8, -9.0, 16.5, 13.2, -0.5, 6.0, 23.2]]
robert covington ['minnesota timberwolves', 'A', 9.2, 0.0, 0.0, 1176, 17.56, 1.1]
['minnesota timberwolves', 'A', 9.2, 0.0, 0.0, 1176, 17.56, 1.1, [20.5, 13.2, 5.8, 29.0, 14.0, 38.0, 4.8, 22.5, 22.0, 0.0, 12.0, 14.5, 17.8, 10.5, 23.0, 17.2, 13.2, 9.0, 14.0, 13.0, 4.0, 19.5, 20.8, 41.5, 21.5]]
robert franks ['charlotte hornets', 'A', 4, 0.0, 0.0, 3005, 0, 0.2]
['charlotte hornets', 'A', 4, 0.0, 0.0, 3005, 0, 0.2, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

['milwaukee bucks', 'G', 5.1, 0.0, 0.0, 1571, 7.79, 0.2, [5.0, -3.0, 0.0, 20.2, 4.0, 12.8, 15.0, 5.0, 21.2, 13.5, 6.8, 9.5, 0.0, 0.0, 0.0, 0.0, 16.0, 10.5, -0.2, 5.5, 0.0, 2.5, 0.0, 1.5, -5.5]]
steven adams ['oklahoma city thunder', 'C', 13.7, 0.0, 0.0, 1064, 25.41, 1.9]
['oklahoma city thunder', 'C', 13.7, 0.0, 0.0, 1064, 25.41, 1.9, [1.8, 13.5, 11.0, 0.0, 33.0, 8.5, 0.0, 24.0, 21.0, 27.0, 36.5, 12.2, 39.2, 40.0, 32.8, 20.0, 24.5, 29.5, 17.0, 23.8, 46.5, 38.5, 17.0, 27.0, 40.2]]
svi mykhailiuk ['detroit pistons', 'G', 4.1, 0.0, 0.0, 1460, 4.89, 0.4]
['detroit pistons', 'G', 4.1, 0.0, 0.0, 1460, 4.89, 0.4, [0.0, 0.0, 0.0, 0.0, 3.0, 5.5, -7.0, -5.0, 1.5, 11.5, 13.8, -2.5, 1.5, 16.0, 4.5, 14.0, -0.5, 0.0, 3.0, 3.0, 14.0, 0.5, 9.8, 6.2, 10.0]]
tj warren ['indiana pacers', 'A', 9, 0.0, 0.0, 1162, 16.72, 0.7]
['indiana pacers', 'A', 9, 0.0, 0.0, 1162, 16.72, 0.7, [0.2, -1.0, 16.8, 2.2, 32.8, 16.5, 20.0, -9.0, 17.0, 5.0, 22.8, 25.5, 22.5, 19.0, 22.2, 24.8, 14.5, 17.8, 10.8, 27.8, 0.8, 15.5, 

['denver nuggets', 'A', 4.7, 0.0, 0.0, 1446, 7.13, 0.3, [21.8, 8.8, 10.8, -2.5, 14.5, 5.5, 10.5, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 1.0, 13.8, 8.0, 5.0, 2.8, 0.0, 6.2, 0.0, 0.0]]
trae young ['atlanta hawks', 'G', 13.6, 0.0, 0.0, 1189, 28.26, 7]
['atlanta hawks', 'G', 13.6, 0.0, 0.0, 1189, 28.26, 7, [43.0, 47.0, 14.8, 0.0, 42.5, 43.0, 31.5, 27.0, 20.5, 9.0, 26.5, 25.0, 34.5, 19.0, 40.2, 39.5, 17.5, 18.0, 23.2, 22.2, 39.5, 2.0, 0.0, 25.0, 39.5]]
tremont waters ['boston celtics', 'G', 4, 0.0, 0.0, 3004, 8.2, 2.6]
['boston celtics', 'G', 4, 0.0, 0.0, 3004, 8.2, 2.6, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 6.5, 0.0, 5.0, 3.5]]
treveon graham ['minnesota timberwolves', 'G', 4, 0.0, 0.0, 1556, 4.97, 0.5]
['minnesota timberwolves', 'G', 4, 0.0, 0.0, 1556, 4.97, 0.5, [17.0, 11.8, 5.5, 8.0, 5.2, 13.2, 7.2, 5.2, -3.5, 0.0, 14.5, 4.5, 8.1, 3.5, 6.0, -3.0, -2.5, -5.5, 8.8, -3.0, 1.8, 1.5, 0.0, 0.0, 0.0]]
trevor ariza ['sa

['sacramento kings', 'G', 4, 0.0, 0.0, 1431, 5.97, 1.7, [0.0, 3.5, 0.0, 0.0, 3.0, 0.0, 17.0, 8.8, 4.0, 0.0, 10.5, -1.2, 8.5, 12.5, 12.5, 11.0, 9.0, 1.0, 5.5, 4.5, 0.0, 2.5, 0.0, 0.0, 1.0]]
yuta watanabe ['memphis grizzlies', 'G', 4, 0.0, 0.0, 1612, -1.25, 1.5]
['memphis grizzlies', 'G', 4, 0.0, 0.0, 1612, -1.25, 1.5, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.0, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
zach lavine ['chicago bulls', 'G', 10.4, 0.0, 0.0, 1106, 19.25, 5.1]
['chicago bulls', 'G', 10.4, 0.0, 0.0, 1106, 19.25, 5.1, [17.8, 41.8, 14.2, 16.0, 10.0, 3.8, 24.0, 18.0, 4.2, 9.5, 10.5, 38.0, 20.8, 27.0, 30.0, 2.5, 40.5, 4.0, 30.0, 33.5, 11.2, 27.5, 8.5, 13.8, 24.2]]
zach norvell jr ['los angeles lakers', 'G', 4, 0.0, 0.0, 2985, 3, 0.6]
['los angeles lakers', 'G', 4, 0.0, 0.0, 2985, 3, 0.6, [0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
zhaire smith ['philadelphia 

In [17]:
columns=['team','role','quotation','pred1','pred2',\
         'id','avg score','popularity',\
         'round scores']
df_final = pd.DataFrame.from_dict(players_with_id_, orient='index', \
                                  columns=columns)
new_columns = ['team','role','quotation','pred1','pred2',\
         'id','avg score','popularity',\
         'round scores']
df_final = df_final[new_columns]
df_final = df_final.loc[:,~df_final.columns.duplicated()]

In [18]:
df_final

,team,role,quotation,pred1,pred2,id,avg score,popularity,round scores
aaron gordon,orlando magic,A,11.7,0.0,0.0,1049,19.12,0.9,"[14.5, 14.5, 5.8, 7.2, 15.0, 22.2, 12.2, 16.8,..."
aaron holiday,indiana pacers,G,7.3,0.0,0.0,1470,14.80,1.5,"[-5.5, 0.0, 0.0, 3.0, 13.0, 17.0, 15.8, 11.8, ..."
abdel nader,oklahoma city thunder,A,4.3,0.0,0.0,1533,7.25,0.8,"[0.0, -4.5, 0.2, 5.0, 0.0, 0.0, 0.0, 5.5, 6.2,..."
admiral schofield,washington wizards,A,4.0,0.0,0.0,2987,3.95,0.9,"[-4.5, 15.5, 0.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0...."
al horford,philadelphia 76ers,C,11.7,0.0,0.0,1060,22.42,1.9,"[17.8, 33.5, 25.0, 35.8, 35.2, 14.0, 30.2, 17...."
...,...,...,...,...,...,...,...,...,...
yuta watanabe,memphis grizzlies,G,4.0,0.0,0.0,1612,-1.25,1.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
zach lavine,chicago bulls,G,10.4,0.0,0.0,1106,19.25,5.1,"[17.8, 41.8, 14.2, 16.0, 10.0, 3.8, 24.0, 18.0..."
zach norvell jr,los angeles lakers,G,4.0,0.0,0.0,2985,3.00,0.6,"[0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
zhaire smith,philadelphia 76ers,G,4.0,0.0,0.0,1403,0.00,0.4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [19]:
def get_trend_delta(avg_score,popularity,quotation,round_scores):
    if len(round_scores) > 0:
        if sum(round_scores) == 0:
            return 0.0
        first_games = round_scores[:-3]
        last_games = round_scores[-4:-1]
        if 0.0 in last_games:
            return 1
        delta = (sum(last_games) / len(last_games)) - (sum(first_games) / len(first_games))
        return delta

def get_hot_index(avg_score,popularity,quotation,round_scores):
    #print(avg_score,popularity,quotation,round_scores)
    if sum(round_scores) == 0:
        return 0.0
    first_games = round_scores[:-3]
    last_games = round_scores[-4:-1]
    if 0.0 in last_games:
        return -999
    else:
        delta = (sum(last_games) / len(last_games)) - (sum(first_games) / len(first_games))
    hot_score = ((0.45 * avg_score) + (0.55 * delta)) * popularity * 0.75 / (quotation * 0.25)
    return hot_score  

In [20]:
df_final["trend delta"] = df_final.apply(
    lambda row: get_trend_delta(row['avg score'], row['popularity'],row['quotation'], row['round scores']), axis=1
)

In [21]:
df_final['hot index'] = ''
df_final["hot index"] = df_final.apply(
    lambda row: get_hot_index(row['avg score'], row['popularity'],row['quotation'], row['round scores']), axis=1
)

In [22]:
df_final = df_final.drop(['pred1','pred2'], axis=1)
df_final = df_final[['team','role','quotation','id',\
                     'avg score','popularity','round scores',\
                     'trend delta','hot index']]
df_final["round scores"] = df_final.apply(
    lambda row: row['round scores'][-4:], axis=1
)
df_final.sort_values(by='hot index', ascending=False).head(50)#.sort_values(by='quotation', ascending=True)

,team,role,quotation,id,avg score,popularity,round scores,trend delta,hot index
luka doncic,dallas mavericks,G,19.5,1090,43.84,24.6,"[70.0, 16.2, 50.0, 59.8]",7.281818,89.820277
jayson tatum,boston celtics,A,12.7,1053,23.72,15.5,"[34.8, 30.5, 13.2, 8.0]",1.566667,42.236890
devonte' graham,charlotte hornets,G,10.5,1502,23.72,10.0,"[25.2, 26.8, 33.0, 20.0]",5.010606,38.370952
bam adebayo,miami heat,C,16.2,2934,34.28,12.8,"[49.0, 23.8, 35.0, 35.5]",1.269697,38.220642
michael porter jr,denver nuggets,A,6.2,1188,10.68,4.8,"[18.2, 29.2, 31.0, 13.8]",20.769697,37.693935
gordon hayward,boston celtics,A,10.9,1107,28.25,6.4,"[22.0, 41.8, 20.8, 20.0]",15.263636,37.180183
lebron james,los angeles lakers,A,22.8,1019,43.65,13.1,"[53.2, 39.0, 42.0, 49.8]",3.051515,36.750384
antetokounmpo thanasis,milwaukee bucks,A,4.0,899,5.14,9.9,"[11.8, 2.0, 1.5, -3.0]",3.481818,31.392900
jordan mcrae,washington wizards,G,5.5,1626,15.13,2.9,"[15.0, 45.5, 24.8, 17.8]",23.483333,31.200309
damion lee,golden state warriors,G,5.6,1519,13.20,5.1,"[24.8, 23.2, 3.2, 8.0]",9.034848,29.805402


In [23]:
cols = list(df_final.columns.values)
cols = ['name'] + cols

In [24]:
# Get names of indexes for which column Age has value 30
df_final['name'] = df_final.index
df_final = df_final[cols]

In [25]:
#show all dataframe
pd.set_option('display.max_rows', df_final.shape[0]+1)


#removing most costly players
#indexNames = df_final[ df_final['quotation'] > 18 ].index
#df_final.drop(indexNames , inplace=True)

#show only players that already played last game
indexNames = df_final[ df_final['round scores'].str[-1] == 0.0 ].index
df_final["round scores"] = df_final.apply(
    lambda row: row['round scores'][-4:], axis=1
)
display(df_final.drop(indexNames).drop('name',axis=1).sort_values(by='hot index', ascending=False))

,team,role,quotation,id,avg score,popularity,round scores,trend delta,hot index
luka doncic,dallas mavericks,G,19.5,1090,43.84,24.6,"[70.0, 16.2, 50.0, 59.8]",7.281818,89.820277
jayson tatum,boston celtics,A,12.7,1053,23.72,15.5,"[34.8, 30.5, 13.2, 8.0]",1.566667,42.236890
devonte' graham,charlotte hornets,G,10.5,1502,23.72,10.0,"[25.2, 26.8, 33.0, 20.0]",5.010606,38.370952
bam adebayo,miami heat,C,16.2,2934,34.28,12.8,"[49.0, 23.8, 35.0, 35.5]",1.269697,38.220642
michael porter jr,denver nuggets,A,6.2,1188,10.68,4.8,"[18.2, 29.2, 31.0, 13.8]",20.769697,37.693935
gordon hayward,boston celtics,A,10.9,1107,28.25,6.4,"[22.0, 41.8, 20.8, 20.0]",15.263636,37.180183
lebron james,los angeles lakers,A,22.8,1019,43.65,13.1,"[53.2, 39.0, 42.0, 49.8]",3.051515,36.750384
antetokounmpo thanasis,milwaukee bucks,A,4.0,899,5.14,9.9,"[11.8, 2.0, 1.5, -3.0]",3.481818,31.392900
jordan mcrae,washington wizards,G,5.5,1626,15.13,2.9,"[15.0, 45.5, 24.8, 17.8]",23.483333,31.200309
damion lee,golden state warriors,G,5.6,1519,13.20,5.1,"[24.8, 23.2, 3.2, 8.0]",9.034848,29.805402


In [26]:
now = datetime.today()
name_folder = '{}_{}_{}/'.format(now.day, now.month, now.year)
prefix = '{}_{}_{}-'.format(now.day, now.month, now.year)
df_final.to_csv('data/' + name_folder + prefix + 'hot_players.csv', sep=',', index=False, header=True)

## Now we have to save also the player stats after they played a game